In [1]:
# 필요한 라이브러리 설치 (필요 시 실행)
# pip install coremltools scikit-learn pandas numpy

import pandas as pd
import numpy as np
import coremltools as ct
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler

# ✅ 1. 데이터 생성 (습관 추천 데이터셋)
num_samples = 200000  # 데이터 크기 조정

np.random.seed(42)
data = {
    "category": np.random.randint(0, 6, num_samples),  # 0~5 (운동, 독서, 명상, 식단, 공부, 취미)
    "time_of_day": np.random.randint(0, 6, num_samples),  # 0~5 (새벽~밤)
    "frequency": np.random.randint(1, 8, num_samples),  # 1~7 (주 몇 회)
    "duration_weeks": np.random.randint(1, 13, num_samples),  # 1~12주 지속
    "completion": np.random.randint(0, 2, num_samples),  # 0(실패) or 1(성공)
    "recommended_habit": np.random.randint(0, 6, num_samples)  # 추천 습관 (0~5 중 하나)
}

df = pd.DataFrame(data)

# ✅ 2. 데이터 정규화 (MinMaxScaler 적용)
X = df.drop(columns=["recommended_habit"]).astype(np.float32)
y = df["recommended_habit"]

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# ✅ 3. 데이터 분할 (80% 훈련, 20% 테스트)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# ✅ 4. 랜덤 포레스트 모델 학습
rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)

# ✅ 5. Core ML 모델로 변환 (최신 방식)
coreml_model = ct.converters.sklearn.convert(rf_model)

# ✅ 6. Core ML 모델 저장
mlmodel_path = "habit_recommendation.mlmodel"
coreml_model.save(mlmodel_path)

print(f"✅ Core ML 모델이 저장되었습니다: {mlmodel_path}")

✅ Core ML 모델이 저장되었습니다: habit_recommendation.mlmodel


In [3]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import onnx
import onnxmltools
import skl2onnx
from skl2onnx.common.data_types import FloatTensorType
import coremltools as ct

# 데이터 샘플 개수
num_samples = 30000  

# 랜덤 데이터 생성
np.random.seed(42)
data = {
    "category": np.random.randint(0, 6, num_samples),
    "time_of_day": np.random.randint(0, 6, num_samples),
    "frequency": np.random.randint(1, 8, num_samples),
    "duration_weeks": np.random.randint(1, 13, num_samples),
    "completion": np.random.randint(0, 2, num_samples),
    "recommended_habit": np.random.randint(0, 6, num_samples)
}

df = pd.DataFrame(data)
df

,category,time_of_day,frequency,duration_weeks,completion,recommended_habit
0,3,0,5,11,1,1
1,4,3,1,10,0,2
2,2,4,3,12,1,0
3,4,2,1,4,0,3
4,4,5,3,9,0,4
...,...,...,...,...,...,...
29995,3,2,1,7,0,1
29996,1,2,5,6,0,0
29997,5,1,1,8,1,1
29998,4,4,7,3,0,0
